In [52]:
# Import libraries

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrix
import datetime
from scipy.stats import chisquare

# Pretty display for notebooks

%matplotlib inline


# Allows the use of display() for DataFrames
from IPython.display import display 

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
weather = pd.read_csv("../asset/weather.csv")
key = pd.read_csv("../asset/key.csv")
submission_example = pd.read_csv("../asset/sampleSubmission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("weather  data : ", weather.shape)
print("key  data : ", key.shape)
print("submission  data : ", submission_example.shape)

coulmns_should_be_integer_type_of_data = ["tmax", "tmin", "tavg", "dewpoint", "wetbulb", "heat", "cool"]
coulmns_should_be_float_type_of_data = ["snowfall", "preciptotal", "stnpressure", "sealevel", "depart",\
                                        "resultspeed", "resultdir", "avgspeed"]

import pickle
import costum_functions as cf

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_weather = weather.copy()
df_key = key.copy()
df_train_with_weather = pd.read_csv("df_train_with_weather_201806251508.csv")

def addDays(date):
    temp = date.split("-")
    year, month, date = int(temp[0]), int(temp[1]), int(temp[2])
    return datetime.datetime(year, month, date).weekday()
df_train_with_weather["day"] = df_train_with_weather["date"].apply(addDays)

Train data :  (4617600, 4)
Test  data :  (526917, 3)
weather  data :  (20517, 20)
key  data :  (45, 2)
submission  data :  (526917, 2)


In [53]:
df_train_with_weather.head()

,date,store_nbr,item_nbr,units,station_nbr,tmax,tmin,tavg,depart,dewpoint,...,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,day
0,2012-01-01,1,1,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
1,2012-01-01,1,2,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
2,2012-01-01,1,3,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
3,2012-01-01,1,4,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
4,2012-01-01,1,5,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6


In [54]:
df_handleNaN = df_train_with_weather

In [55]:
df_handleNaN.head()

,date,store_nbr,item_nbr,units,station_nbr,tmax,tmin,tavg,depart,dewpoint,...,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,day
0,2012-01-01,1,1,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
1,2012-01-01,1,2,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
2,2012-01-01,1,3,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
3,2012-01-01,1,4,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
4,2012-01-01,1,5,0,1,52,31,42,M,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6


In [56]:
col_1  = list(df_handleNaN.columns)

In [57]:
def handle_NaN(a):
    temp = str(a).strip()
    if temp == "T" or temp == "-":
        return 0
    elif temp == "M" :
        return np.nan
    else :
        return int(a)

In [58]:
df_handleNaN["tavg"] = df_handleNaN["tavg"].apply(handle_NaN)
df_handleNaN["tmin"] = df_handleNaN["tmin"].apply(handle_NaN)
df_handleNaN["tmax"] = df_handleNaN["tmax"].apply(handle_NaN)
df_handleNaN["depart"] = df_handleNaN["depart"].apply(handle_NaN)

In [59]:
df_handleNaN.head()

,date,store_nbr,item_nbr,units,station_nbr,tmax,tmin,tavg,depart,dewpoint,...,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,day
0,2012-01-01,1,1,0,1,52.0,31.0,42.0,NaN,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
1,2012-01-01,1,2,0,1,52.0,31.0,42.0,NaN,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
2,2012-01-01,1,3,0,1,52.0,31.0,42.0,NaN,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
3,2012-01-01,1,4,0,1,52.0,31.0,42.0,NaN,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
4,2012-01-01,1,5,0,1,52.0,31.0,42.0,NaN,36,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,6
